# MOFs LM Probing for isMOF

In [1]:
from baseline import PromptSet

from tqdm.auto import tqdm
from nltk.corpus import stopwords

import pandas as pd

from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline, logging

logging.set_verbosity_error()  # avoid irritating transformers warnings

## Read in the Triples

In [2]:
file = "../data/mof_synonyms.csv"

In [3]:
df = pd.read_csv(file)
df.head()

,SubjectEntity,Relation,ObjectEntity
0,MROF-1,isTypeOf,"MOF, Metal-Organic Framework"
1,MOF-CJ3,isTypeOf,"MOF, Metal-Organic Framework"
2,"catena-(octakis(μ6-4,4',4''-s-Triazine-2,4,6-t...",isTypeOf,"MOF, Metal-Organic Framework"
3,Spirof-MOF,isTypeOf,"MOF, Metal-Organic Framework"
4,NJU-Bai9,isTypeOf,"MOF, Metal-Organic Framework"


In [4]:
df.shape

(1800, 3)

In [5]:
df['ObjectEntity'] = 'MOF, Metal-Organic'

In [6]:
df.head()

,SubjectEntity,Relation,ObjectEntity
0,MROF-1,isTypeOf,"MOF, Metal-Organic"
1,MOF-CJ3,isTypeOf,"MOF, Metal-Organic"
2,"catena-(octakis(μ6-4,4',4''-s-Triazine-2,4,6-t...",isTypeOf,"MOF, Metal-Organic"
3,Spirof-MOF,isTypeOf,"MOF, Metal-Organic"
4,NJU-Bai9,isTypeOf,"MOF, Metal-Organic"


## Probing BERT

In [7]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-cased")
model = AutoModelForMaskedLM.from_pretrained("bert-large-cased")

In [8]:
mask_token = tokenizer.mask_token
mask_token

'[MASK]'

## Creat the Prompt Pipeline

In [9]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Create Prompts

In [10]:
def create_type_prompt1(sub, mask):
    return f"{sub} is an {mask} material."
    
def create_type_prompt2(sub, mask):
    return f"{sub} is an {mask} structure."

def create_type_prompt3(sub, mask):
    return f"{sub} is an {mask}."

def create_type_prompt4(sub, mask):
    return f"{sub} is an {mask} made of metal center and organic linkers."
    
def create_type_prompt5(sub, mask):
    return f"{sub} is an {mask} material containing metal center and organic linkers."
    
def create_type_prompt6(sub, mask):
    return f"{sub} is an {mask} structure with metal center and organic linkers." 
    
def create_type_prompt7(sub, mask):
    return f"{sub} is an type of {mask} material."
    
def create_type_prompt8(sub, mask):
    return f"{sub} is in the class of {mask} material."
    
def create_type_prompt9(sub, mask):
    return f"{sub} has SBUs and organic linkers. The type of {sub} is {mask}."
    
def create_type_prompt10(sub, mask):
    return f"{sub} is a type of polymer with metal clusters and organic linkers. {sub} is an {mask} framework."
    
def create_type_prompt11(sub, mask):
    return f"{sub} is an type of {mask}."
    
def create_type_prompt12(sub, mask):
    return f"The porous structure of {sub} makes it an {mask}."

In [11]:
processors = [create_type_prompt1, create_type_prompt2,
              create_type_prompt3, create_type_prompt4,
              create_type_prompt5, create_type_prompt6,
              create_type_prompt7, create_type_prompt8,
              create_type_prompt9, create_type_prompt10,
              create_type_prompt11, create_type_prompt12
             ]

## Test Small

In [4]:
processors = [create_type_prompt1, create_type_prompt2, create_type_prompt3]

In [5]:
processors[0]('MOF-1', '[MASK]')

'MOF-1 is an [MASK] material.'

In [12]:
mofs_sub =df[:10]

In [16]:
prompts = PromptSet([processors[0](row.SubjectEntity, mask_token) for _, row in mofs_sub.iterrows()])

In [17]:
prompts.__getitem__(9)

"catena-[(μ8-4,4'-(Hexafluoroisopropylidene)diphthalato)-aqua-di-cadmium(ii) hemihydrate] is an [MASK] material."

In [18]:
for i in prompts:
    print(i)

MROF-1 is an [MASK] material.
MOF-CJ3 is an [MASK] material.
catena-(octakis(μ6-4,4',4''-s-Triazine-2,4,6-triyltribenzoato)-dodeca-aqua-dodeca-copper dimethylsulfoxide solvate hydrate)|PCN-6|Teaching Subset: MOFs is an [MASK] material.
Spirof-MOF is an [MASK] material.
NJU-Bai9 is an [MASK] material.
MOF-1 is an [MASK] material.
MOF-2 is an [MASK] material.
TCM-9-Ho is an [MASK] material.
catena-[bis(μ4-(2RS,3RS)-2,3-Dihydroxybutanedioato-O,O',O'',O''',O'''',O''''')-(μ2-oxalato-O,O',O'',O''')-diaqua-di-lanthanum(iii) tetrahydrate] is an [MASK] material.
catena-[(μ8-4,4'-(Hexafluoroisopropylidene)diphthalato)-aqua-di-cadmium(ii) hemihydrate] is an [MASK] material.


In [21]:
# Run the model
outputs = []
for out in tqdm(pipe(prompts, batch_size=8), total=len(prompts)):
    outputs.append(out)

  0%|          | 0/10 [00:00<?, ?it/s]

In [23]:
outputs

[[{'score': 0.24670414626598358,
   'token': 7878,
   'token_str': 'organic',
   'sequence': 'MROF - 1 is an organic material.'},
  {'score': 0.0834609642624855,
   'token': 6700,
   'token_str': 'experimental',
   'sequence': 'MROF - 1 is an experimental material.'},
  {'score': 0.08127032965421677,
   'token': 16419,
   'token_str': 'engineered',
   'sequence': 'MROF - 1 is an engineered material.'},
  {'score': 0.06450482457876205,
   'token': 10312,
   'token_str': 'optical',
   'sequence': 'MROF - 1 is an optical material.'},
  {'score': 0.05137038230895996,
   'token': 2327,
   'token_str': 'active',
   'sequence': 'MROF - 1 is an active material.'},
  {'score': 0.04575609043240547,
   'token': 12154,
   'token_str': 'explosive',
   'sequence': 'MROF - 1 is an explosive material.'},
  {'score': 0.04066336899995804,
   'token': 3682,
   'token_str': 'advanced',
   'sequence': 'MROF - 1 is an advanced material.'},
  {'score': 0.018374163657426834,
   'token': 25460,
   'token_str':

In [22]:
results = []
for idx, row in mofs_sub.iterrows():
    for output in outputs[idx]:
        record = {}
        record['SubjectEntity'] = row['SubjectEntity']
        record['Relation'] = row['Relation']
        record['ObjectEntity'] = row['ObjectEntity']
        record['prompt']= prompts[idx]
        record['predictedScore'] = output['score']
        record['predictedToken'] = output['token_str']
        results.append(record)

In [23]:
predicted_df = pd.DataFrame(results)
predicted_df

,SubjectEntity,Relation,ObjectEntity,prompt,predictedScore,predictedToken
0,MROF-1,isTypeOf,"MOF, Metal-Organic Framework",MROF-1 is an [MASK] material.,0.246704,organic
1,MROF-1,isTypeOf,"MOF, Metal-Organic Framework",MROF-1 is an [MASK] material.,0.083461,experimental
2,MROF-1,isTypeOf,"MOF, Metal-Organic Framework",MROF-1 is an [MASK] material.,0.081270,engineered
3,MROF-1,isTypeOf,"MOF, Metal-Organic Framework",MROF-1 is an [MASK] material.,0.064505,optical
4,MROF-1,isTypeOf,"MOF, Metal-Organic Framework",MROF-1 is an [MASK] material.,0.051370,active
...,...,...,...,...,...,...
995,"catena-[(μ8-4,4'-(Hexafluoroisopropylidene)dip...",isTypeOf,"MOF, Metal-Organic Framework","catena-[(μ8-4,4'-(Hexafluoroisopropylidene)dip...",0.000374,emission
996,"catena-[(μ8-4,4'-(Hexafluoroisopropylidene)dip...",isTypeOf,"MOF, Metal-Organic Framework","catena-[(μ8-4,4'-(Hexafluoroisopropylidene)dip...",0.000372,ceramics
997,"catena-[(μ8-4,4'-(Hexafluoroisopropylidene)dip...",isTypeOf,"MOF, Metal-Organic Framework","catena-[(μ8-4,4'-(Hexafluoroisopropylidene)dip...",0.000353,polymer
998,"catena-[(μ8-4,4'-(Hexafluoroisopropylidene)dip...",isTypeOf,"MOF, Metal-Organic Framework","catena-[(μ8-4,4'-(Hexafluoroisopropylidene)dip...",0.000353,acid


### Evaluate on Small

In [26]:
predicted_df.loc[10]['ObjectEntity'].split(",")

['MOF', ' Metal-Organic Framework']

In [24]:
# Compute a column indicating whether the predictedToken in the ObjectEntity list
def hasPredicted(row):
    token = row['predictedToken']
    objectEntities = row['ObjectEntity']
    #print(objectEntities)
    if pd.notna(objectEntities):
        types = objectEntities.split(",") # get individual types
        for tp in types: # for each type
            type_words = tp.lower().split() # get the individual words in the type
            if token.lower() in type_words: # check if a type word was predicted
                return 1
       
    return 0

In [49]:
# test what the function does
def hasPredicted_demo(objectEntities, token):
    #print(objectEntities)
    if pd.notna(objectEntities):
        types = objectEntities.split(",") # get individual types
        for tp in types: # for each type
            type_words = tp.lower().split() # get the individual words in the type
            print(type_words)
            if token.lower() in type_words: # check if a type word was predicted
                return 1
       
    return 0

In [45]:
hasPredicted_demo('MOF, Metal-Organic Framework', 'Framework')

['mof']
['metal-organic', 'framework']


1

In [25]:
predicted_df['predicted'] = predicted_df.apply(hasPredicted, axis=1)

In [26]:
predicted_df.head()

,SubjectEntity,Relation,ObjectEntity,prompt,predictedScore,predictedToken,predicted
0,MROF-1,isTypeOf,"MOF, Metal-Organic Framework",MROF-1 is an [MASK] material.,0.246704,organic,0
1,MROF-1,isTypeOf,"MOF, Metal-Organic Framework",MROF-1 is an [MASK] material.,0.083461,experimental,0
2,MROF-1,isTypeOf,"MOF, Metal-Organic Framework",MROF-1 is an [MASK] material.,0.081270,engineered,0
3,MROF-1,isTypeOf,"MOF, Metal-Organic Framework",MROF-1 is an [MASK] material.,0.064505,optical,0
4,MROF-1,isTypeOf,"MOF, Metal-Organic Framework",MROF-1 is an [MASK] material.,0.051370,active,0


In [27]:
predicted_df['predicted'].sum()

0

In [54]:
predicted_df[predicted_df.predicted == 1]

,SubjectEntity,Relation,ObjectEntity,prompt,predictedScore,predictedToken,predicted


In [32]:
# return the top-k rows based on predictedScore
def top(df, col, n=10):
    return df.sort_values(by=col, ascending=False)[:n]

In [33]:
topk_df = predicted_df.groupby('SubjectEntity', as_index=False).apply(top, col='predictedScore', n=100)

In [34]:
# check whether the predicted is 1 for each group
def predictedOne(df, col):
    total = df[col].sum()
    if total > 0:
        return 1
    else:
        return 0

In [35]:
topk_agg_df = topk_df.groupby('SubjectEntity', as_index=False).apply(predictedOne, col='predicted')

In [36]:
topk_agg_df.columns = ['SubjectEntity', 'predicted']
topk_agg_df

,SubjectEntity,predicted
0,MOF-1,0
1,MOF-2,0
2,MOF-CJ3,0
3,MROF-1,0
4,NJU-Bai9,0
5,Spirof-MOF,0
6,TCM-9-Ho,0
7,"catena-(octakis(μ6-4,4',4''-s-Triazine-2,4,6-t...",0
8,"catena-[(μ8-4,4'-(Hexafluoroisopropylidene)dip...",0
9,"catena-[bis(μ4-(2RS,3RS)-2,3-Dihydroxybutanedi...",0


In [37]:
topk_agg_df.predicted.mean()

0.0

## Test BERT

In [12]:
def prompt_probe(model, tokenizer, prompt_processor, df_sub, mask_token, \
                 subjectCol, relationCol, objectCol, top_k=100):

    pipe = pipeline(
        task="fill-mask",
        model=model,
        tokenizer=tokenizer,
        top_k=top_k,
    )

    prompts = PromptSet([prompt_processor(row.SubjectEntity, mask_token) for _, row in df_sub.iterrows()])

    outputs = []
    for out in tqdm(pipe(prompts, batch_size=8), total=len(prompts)):
        outputs.append(out)

    results = []
    for idx, row in df_sub.iterrows():
        for output in outputs[idx]:
            record = {}
            record['SubjectEntity'] = row[subjectCol]
            record['Relation'] = row[relationCol]
            record['ObjectEntity'] = row[objectCol]
            record['prompt']= prompts[idx]
            record['predictedScore'] = output['score']
            record['predictedToken'] = output['token_str']
            results.append(record)

    return pd.DataFrame(results)

In [13]:
# Compute a column indicating whether the predictedToken in the ObjectEntity list
def hasPredicted(row):
    token = row['predictedToken']
    objectEntities = row['ObjectEntity']
    #print(objectEntities)
    if pd.notna(objectEntities):
        types = objectEntities.split(",") # get individual types
        for tp in types: # for each type
            type_words = tp.lower().split() # get the individual words in the type
            if token.lower() in type_words: # check if a type word was predicted
                return 1
       
    return 0

In [14]:
# return the top-k rows based on predictedScore
def top(df, col, n=10):
    return df.sort_values(by=col, ascending=False)[:n]

In [15]:
# check whether the predicted is 1 for each group
def predictedOne(df, col):
    total = df[col].sum()
    if total > 0:
        return 1
    else:
        return 0

In [16]:
# Assume the predicted_df has a binary column 'predicted' indicating whether ObjectEntity has been predicted
# 1 for yes, 0 for no
def evaluate_acc_n(predicted_df, relation, prompt_template, at_k=5):
    topk_df = predicted_df.groupby('SubjectEntity', as_index=False).apply(top, col='predictedScore', n=at_k)
    topk_agg_df = topk_df.groupby('SubjectEntity', as_index=False).apply(predictedOne, col='predicted')
    topk_agg_df.columns = ['SubjectEntity', 'predicted']
    result = {}
    result['Relation'] = relation
    result['prompt_template'] = prompt_template
    result['acc'] = topk_agg_df.predicted.mean()
    result['at_k'] = at_k

    return result

In [17]:
pd.set_option('display.max_colwidth', None)

In [18]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntity', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasMetals', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

...

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

CPU times: user 3h 46min 59s, sys: 2min 46s, total: 3h 49min 45s
Wall time: 18min 6s


In [19]:
hasType_df = pd.DataFrame(acc_list)

In [20]:
hasType_df.sample(10)

,Relation,prompt_template,acc,at_k
9,hasMetals,{SubjectEntity} is an [MASK] structure.,0.0,20
13,hasMetals,{SubjectEntity} is an [MASK].,0.0,5
43,hasMetals,{SubjectEntity} is in the class of [MASK] material.,0.0,5
44,hasMetals,{SubjectEntity} is in the class of [MASK] material.,0.0,10
17,hasMetals,{SubjectEntity} is an [MASK].,0.0,100
35,hasMetals,{SubjectEntity} is an [MASK] structure with metal center and organic linkers.,0.0,100
28,hasMetals,{SubjectEntity} is an [MASK] material containing metal center and organic linkers.,0.0,50
15,hasMetals,{SubjectEntity} is an [MASK].,0.0,20
38,hasMetals,{SubjectEntity} is an type of [MASK] material.,0.0,10
26,hasMetals,{SubjectEntity} is an [MASK] material containing metal center and organic linkers.,0.0,10


In [21]:
result_df.to_csv("../data/predicted_hasType_BERT.csv", index=None)

In [22]:
hasType_df.to_csv("../data/predicted_hasType_BERT_eval.csv", index=None)

In [23]:
hasType_df[hasType_df['at_k'] == 1].sort_values('acc', ascending=False)

,Relation,prompt_template,acc,at_k
0,hasMetals,{SubjectEntity} is an [MASK] material.,0.0,1
6,hasMetals,{SubjectEntity} is an [MASK] structure.,0.0,1
12,hasMetals,{SubjectEntity} is an [MASK].,0.0,1
18,hasMetals,{SubjectEntity} is an [MASK] made of metal center and organic linkers.,0.0,1
24,hasMetals,{SubjectEntity} is an [MASK] material containing metal center and organic linkers.,0.0,1
30,hasMetals,{SubjectEntity} is an [MASK] structure with metal center and organic linkers.,0.0,1
36,hasMetals,{SubjectEntity} is an type of [MASK] material.,0.0,1
42,hasMetals,{SubjectEntity} is in the class of [MASK] material.,0.0,1
48,hasMetals,{SubjectEntity} has SBUs and organic linkers. The type of {SubjectEntity} is [MASK].,0.0,1
54,hasMetals,{SubjectEntity} is a type of polymer with metal clusters and organic linkers. {SubjectEntity} is an [MASK] framework.,0.0,1


In [24]:
hasType_df.sort_values('acc', ascending=False)

,Relation,prompt_template,acc,at_k
0,hasMetals,{SubjectEntity} is an [MASK] material.,0.0,1
1,hasMetals,{SubjectEntity} is an [MASK] material.,0.0,5
52,hasMetals,{SubjectEntity} has SBUs and organic linkers. The type of {SubjectEntity} is [MASK].,0.0,50
51,hasMetals,{SubjectEntity} has SBUs and organic linkers. The type of {SubjectEntity} is [MASK].,0.0,20
50,hasMetals,{SubjectEntity} has SBUs and organic linkers. The type of {SubjectEntity} is [MASK].,0.0,10
...,...,...,...,...
23,hasMetals,{SubjectEntity} is an [MASK] made of metal center and organic linkers.,0.0,100
22,hasMetals,{SubjectEntity} is an [MASK] made of metal center and organic linkers.,0.0,50
21,hasMetals,{SubjectEntity} is an [MASK] made of metal center and organic linkers.,0.0,20
20,hasMetals,{SubjectEntity} is an [MASK] made of metal center and organic linkers.,0.0,10


## Probing MatBERT

In [25]:
# MatBERT
tokenizer = AutoTokenizer.from_pretrained("../model/matbert-base-cased")
model = AutoModelForMaskedLM.from_pretrained("../model/matbert-base-cased")

In [26]:
mask_token = tokenizer.mask_token
mask_token

'[MASK]'

## Creat the Prompt Pipeline

In [27]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Run

In [28]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntity', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasMetals', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

...

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

CPU times: user 1h 22min 4s, sys: 54.9 s, total: 1h 22min 58s
Wall time: 7min 37s


In [29]:
hasType_df = pd.DataFrame(acc_list)

In [30]:
hasType_df.sample(10)

,Relation,prompt_template,acc,at_k
23,hasMetals,{SubjectEntity} is an [MASK] made of metal center and organic linkers.,1.000000,100
66,hasMetals,The porous structure of {SubjectEntity} makes it an [MASK].,0.000000,1
6,hasMetals,{SubjectEntity} is an [MASK] structure.,0.000000,1
47,hasMetals,{SubjectEntity} is in the class of [MASK] material.,0.154030,100
34,hasMetals,{SubjectEntity} is an [MASK] structure with metal center and organic linkers.,0.978452,50
33,hasMetals,{SubjectEntity} is an [MASK] structure with metal center and organic linkers.,0.760575,20
11,hasMetals,{SubjectEntity} is an [MASK] structure.,0.094972,100
54,hasMetals,{SubjectEntity} is a type of polymer with metal clusters and organic linkers. {SubjectEntity} is an [MASK] framework.,0.000000,1
52,hasMetals,{SubjectEntity} has SBUs and organic linkers. The type of {SubjectEntity} is [MASK].,0.000000,50
38,hasMetals,{SubjectEntity} is an type of [MASK] material.,0.033520,10


In [31]:
result_df.to_csv("../data/predicted_hasType_MatBERT.csv", index=None)

In [32]:
hasType_df.to_csv("../data/predicted_hasType_MatBERT_eval.csv", index=None)

In [33]:
hasType_df[hasType_df['at_k'] == 1].sort_values('acc', ascending=False)

,Relation,prompt_template,acc,at_k
18,hasMetals,{SubjectEntity} is an [MASK] made of metal center and organic linkers.,0.684757,1
60,hasMetals,{SubjectEntity} is an type of [MASK].,0.007981,1
0,hasMetals,{SubjectEntity} is an [MASK] material.,0.000000,1
6,hasMetals,{SubjectEntity} is an [MASK] structure.,0.000000,1
12,hasMetals,{SubjectEntity} is an [MASK].,0.000000,1
24,hasMetals,{SubjectEntity} is an [MASK] material containing metal center and organic linkers.,0.000000,1
30,hasMetals,{SubjectEntity} is an [MASK] structure with metal center and organic linkers.,0.000000,1
36,hasMetals,{SubjectEntity} is an type of [MASK] material.,0.000000,1
42,hasMetals,{SubjectEntity} is in the class of [MASK] material.,0.000000,1
48,hasMetals,{SubjectEntity} has SBUs and organic linkers. The type of {SubjectEntity} is [MASK].,0.000000,1


In [34]:
hasType_df.sort_values('acc', ascending=False)

,Relation,prompt_template,acc,at_k
23,hasMetals,{SubjectEntity} is an [MASK] made of metal center and organic linkers.,1.000000,100
22,hasMetals,{SubjectEntity} is an [MASK] made of metal center and organic linkers.,0.999202,50
29,hasMetals,{SubjectEntity} is an [MASK] material containing metal center and organic linkers.,0.996808,100
21,hasMetals,{SubjectEntity} is an [MASK] made of metal center and organic linkers.,0.995211,20
28,hasMetals,{SubjectEntity} is an [MASK] material containing metal center and organic linkers.,0.994413,50
...,...,...,...,...
42,hasMetals,{SubjectEntity} is in the class of [MASK] material.,0.000000,1
48,hasMetals,{SubjectEntity} has SBUs and organic linkers. The type of {SubjectEntity} is [MASK].,0.000000,1
49,hasMetals,{SubjectEntity} has SBUs and organic linkers. The type of {SubjectEntity} is [MASK].,0.000000,5
50,hasMetals,{SubjectEntity} has SBUs and organic linkers. The type of {SubjectEntity} is [MASK].,0.000000,10


## Probing MatSciBERT

In [35]:
# MatSciBERT
tokenizer = AutoTokenizer.from_pretrained('m3rg-iitd/matscibert')
model = AutoModelForMaskedLM.from_pretrained('m3rg-iitd/matscibert')

In [36]:
mask_token = tokenizer.mask_token
mask_token

'[MASK]'

## Creat the Prompt Pipeline

In [37]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Run

In [38]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntity', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasMetals', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

...

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

CPU times: user 1h 24min 41s, sys: 12.9 s, total: 1h 24min 53s
Wall time: 7min 45s


In [39]:
hasType_df = pd.DataFrame(acc_list)

In [40]:
hasType_df.sample(10)

,Relation,prompt_template,acc,at_k
59,hasMetals,{SubjectEntity} is a type of polymer with metal clusters and organic linkers. {SubjectEntity} is an [MASK] framework.,0.0,100
20,hasMetals,{SubjectEntity} is an [MASK] made of metal center and organic linkers.,0.0,10
21,hasMetals,{SubjectEntity} is an [MASK] made of metal center and organic linkers.,0.0,20
36,hasMetals,{SubjectEntity} is an type of [MASK] material.,0.0,1
48,hasMetals,{SubjectEntity} has SBUs and organic linkers. The type of {SubjectEntity} is [MASK].,0.0,1
7,hasMetals,{SubjectEntity} is an [MASK] structure.,0.0,5
38,hasMetals,{SubjectEntity} is an type of [MASK] material.,0.0,10
64,hasMetals,{SubjectEntity} is an type of [MASK].,0.0,50
30,hasMetals,{SubjectEntity} is an [MASK] structure with metal center and organic linkers.,0.0,1
51,hasMetals,{SubjectEntity} has SBUs and organic linkers. The type of {SubjectEntity} is [MASK].,0.0,20


In [41]:
result_df.to_csv("../data/predicted_hasType_MatSciBERT.csv", index=None)

In [42]:
hasType_df.to_csv("../data/predicted_hasType_MatSciBERT_eval.csv", index=None)

In [43]:
hasType_df[hasType_df['at_k'] == 1].sort_values('acc', ascending=False)

,Relation,prompt_template,acc,at_k
0,hasMetals,{SubjectEntity} is an [MASK] material.,0.0,1
6,hasMetals,{SubjectEntity} is an [MASK] structure.,0.0,1
12,hasMetals,{SubjectEntity} is an [MASK].,0.0,1
18,hasMetals,{SubjectEntity} is an [MASK] made of metal center and organic linkers.,0.0,1
24,hasMetals,{SubjectEntity} is an [MASK] material containing metal center and organic linkers.,0.0,1
30,hasMetals,{SubjectEntity} is an [MASK] structure with metal center and organic linkers.,0.0,1
36,hasMetals,{SubjectEntity} is an type of [MASK] material.,0.0,1
42,hasMetals,{SubjectEntity} is in the class of [MASK] material.,0.0,1
48,hasMetals,{SubjectEntity} has SBUs and organic linkers. The type of {SubjectEntity} is [MASK].,0.0,1
54,hasMetals,{SubjectEntity} is a type of polymer with metal clusters and organic linkers. {SubjectEntity} is an [MASK] framework.,0.0,1


In [44]:
hasType_df.sort_values('acc', ascending=False)

,Relation,prompt_template,acc,at_k
0,hasMetals,{SubjectEntity} is an [MASK] material.,0.0,1
1,hasMetals,{SubjectEntity} is an [MASK] material.,0.0,5
52,hasMetals,{SubjectEntity} has SBUs and organic linkers. The type of {SubjectEntity} is [MASK].,0.0,50
51,hasMetals,{SubjectEntity} has SBUs and organic linkers. The type of {SubjectEntity} is [MASK].,0.0,20
50,hasMetals,{SubjectEntity} has SBUs and organic linkers. The type of {SubjectEntity} is [MASK].,0.0,10
...,...,...,...,...
23,hasMetals,{SubjectEntity} is an [MASK] made of metal center and organic linkers.,0.0,100
22,hasMetals,{SubjectEntity} is an [MASK] made of metal center and organic linkers.,0.0,50
21,hasMetals,{SubjectEntity} is an [MASK] made of metal center and organic linkers.,0.0,20
20,hasMetals,{SubjectEntity} is an [MASK] made of metal center and organic linkers.,0.0,10


## Probing SciBERT

In [45]:
# SciBERT
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_cased')
model = AutoModelForMaskedLM.from_pretrained('allenai/scibert_scivocab_cased')

In [46]:
mask_token = tokenizer.mask_token
mask_token

'[MASK]'

## Creat the Prompt Pipeline

In [47]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Run

In [48]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntity', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasMetals', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

...

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

CPU times: user 1h 26min 12s, sys: 11.1 s, total: 1h 26min 23s
Wall time: 7min 52s


In [49]:
hasType_df = pd.DataFrame(acc_list)

In [50]:
hasType_df.sample(10)

,Relation,prompt_template,acc,at_k
34,hasMetals,{SubjectEntity} is an [MASK] structure with metal center and organic linkers.,0.0,50
30,hasMetals,{SubjectEntity} is an [MASK] structure with metal center and organic linkers.,0.0,1
49,hasMetals,{SubjectEntity} has SBUs and organic linkers. The type of {SubjectEntity} is [MASK].,0.0,5
63,hasMetals,{SubjectEntity} is an type of [MASK].,0.0,20
53,hasMetals,{SubjectEntity} has SBUs and organic linkers. The type of {SubjectEntity} is [MASK].,0.0,100
28,hasMetals,{SubjectEntity} is an [MASK] material containing metal center and organic linkers.,0.0,50
12,hasMetals,{SubjectEntity} is an [MASK].,0.0,1
43,hasMetals,{SubjectEntity} is in the class of [MASK] material.,0.0,5
23,hasMetals,{SubjectEntity} is an [MASK] made of metal center and organic linkers.,0.0,100
58,hasMetals,{SubjectEntity} is a type of polymer with metal clusters and organic linkers. {SubjectEntity} is an [MASK] framework.,0.0,50


In [51]:
result_df.to_csv("../data/predicted_hasType_SciBERT.csv", index=None)

In [52]:
hasType_df.to_csv("../data/predicted_hasType_SciBERT_eval.csv", index=None)

In [53]:
hasType_df[hasType_df['at_k'] == 1].sort_values('acc', ascending=False)

,Relation,prompt_template,acc,at_k
0,hasMetals,{SubjectEntity} is an [MASK] material.,0.0,1
6,hasMetals,{SubjectEntity} is an [MASK] structure.,0.0,1
12,hasMetals,{SubjectEntity} is an [MASK].,0.0,1
18,hasMetals,{SubjectEntity} is an [MASK] made of metal center and organic linkers.,0.0,1
24,hasMetals,{SubjectEntity} is an [MASK] material containing metal center and organic linkers.,0.0,1
30,hasMetals,{SubjectEntity} is an [MASK] structure with metal center and organic linkers.,0.0,1
36,hasMetals,{SubjectEntity} is an type of [MASK] material.,0.0,1
42,hasMetals,{SubjectEntity} is in the class of [MASK] material.,0.0,1
48,hasMetals,{SubjectEntity} has SBUs and organic linkers. The type of {SubjectEntity} is [MASK].,0.0,1
54,hasMetals,{SubjectEntity} is a type of polymer with metal clusters and organic linkers. {SubjectEntity} is an [MASK] framework.,0.0,1


In [54]:
hasType_df.sort_values('acc', ascending=False)

,Relation,prompt_template,acc,at_k
0,hasMetals,{SubjectEntity} is an [MASK] material.,0.0,1
1,hasMetals,{SubjectEntity} is an [MASK] material.,0.0,5
52,hasMetals,{SubjectEntity} has SBUs and organic linkers. The type of {SubjectEntity} is [MASK].,0.0,50
51,hasMetals,{SubjectEntity} has SBUs and organic linkers. The type of {SubjectEntity} is [MASK].,0.0,20
50,hasMetals,{SubjectEntity} has SBUs and organic linkers. The type of {SubjectEntity} is [MASK].,0.0,10
...,...,...,...,...
23,hasMetals,{SubjectEntity} is an [MASK] made of metal center and organic linkers.,0.0,100
22,hasMetals,{SubjectEntity} is an [MASK] made of metal center and organic linkers.,0.0,50
21,hasMetals,{SubjectEntity} is an [MASK] made of metal center and organic linkers.,0.0,20
20,hasMetals,{SubjectEntity} is an [MASK] made of metal center and organic linkers.,0.0,10


## Probing RoBERTa

In [55]:
# RoBERTa
tokenizer = AutoTokenizer.from_pretrained("roberta-large")
model = AutoModelForMaskedLM.from_pretrained("roberta-large")

In [56]:
mask_token = tokenizer.mask_token
mask_token

'<mask>'

## Creat the Prompt Pipeline

In [57]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Run

In [58]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntity', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasMetals', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

...

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

  0%|          | 0/1800 [00:00<?, ?it/s]

CPU times: user 3h 44min 43s, sys: 6.81 s, total: 3h 44min 50s
Wall time: 17min 43s


In [59]:
hasType_df = pd.DataFrame(acc_list)

In [60]:
hasType_df.sample(10)

,Relation,prompt_template,acc,at_k
29,hasMetals,{SubjectEntity} is an [MASK] material containing metal center and organic linkers.,0.0,100
24,hasMetals,{SubjectEntity} is an [MASK] material containing metal center and organic linkers.,0.0,1
12,hasMetals,{SubjectEntity} is an [MASK].,0.0,1
62,hasMetals,{SubjectEntity} is an type of [MASK].,0.0,10
34,hasMetals,{SubjectEntity} is an [MASK] structure with metal center and organic linkers.,0.0,50
8,hasMetals,{SubjectEntity} is an [MASK] structure.,0.0,10
30,hasMetals,{SubjectEntity} is an [MASK] structure with metal center and organic linkers.,0.0,1
2,hasMetals,{SubjectEntity} is an [MASK] material.,0.0,10
21,hasMetals,{SubjectEntity} is an [MASK] made of metal center and organic linkers.,0.0,20
4,hasMetals,{SubjectEntity} is an [MASK] material.,0.0,50


In [61]:
result_df.to_csv("../data/predicted_hasType_RoBERTa.csv", index=None)

In [62]:
hasType_df.to_csv("../data/predicted_hasType_RoBERTa_eval.csv", index=None)

In [63]:
hasType_df[hasType_df['at_k'] == 1].sort_values('acc', ascending=False)

,Relation,prompt_template,acc,at_k
0,hasMetals,{SubjectEntity} is an [MASK] material.,0.0,1
6,hasMetals,{SubjectEntity} is an [MASK] structure.,0.0,1
12,hasMetals,{SubjectEntity} is an [MASK].,0.0,1
18,hasMetals,{SubjectEntity} is an [MASK] made of metal center and organic linkers.,0.0,1
24,hasMetals,{SubjectEntity} is an [MASK] material containing metal center and organic linkers.,0.0,1
30,hasMetals,{SubjectEntity} is an [MASK] structure with metal center and organic linkers.,0.0,1
36,hasMetals,{SubjectEntity} is an type of [MASK] material.,0.0,1
42,hasMetals,{SubjectEntity} is in the class of [MASK] material.,0.0,1
48,hasMetals,{SubjectEntity} has SBUs and organic linkers. The type of {SubjectEntity} is [MASK].,0.0,1
54,hasMetals,{SubjectEntity} is a type of polymer with metal clusters and organic linkers. {SubjectEntity} is an [MASK] framework.,0.0,1


In [64]:
hasType_df.sort_values('acc', ascending=False)

,Relation,prompt_template,acc,at_k
0,hasMetals,{SubjectEntity} is an [MASK] material.,0.0,1
1,hasMetals,{SubjectEntity} is an [MASK] material.,0.0,5
52,hasMetals,{SubjectEntity} has SBUs and organic linkers. The type of {SubjectEntity} is [MASK].,0.0,50
51,hasMetals,{SubjectEntity} has SBUs and organic linkers. The type of {SubjectEntity} is [MASK].,0.0,20
50,hasMetals,{SubjectEntity} has SBUs and organic linkers. The type of {SubjectEntity} is [MASK].,0.0,10
...,...,...,...,...
23,hasMetals,{SubjectEntity} is an [MASK] made of metal center and organic linkers.,0.0,100
22,hasMetals,{SubjectEntity} is an [MASK] made of metal center and organic linkers.,0.0,50
21,hasMetals,{SubjectEntity} is an [MASK] made of metal center and organic linkers.,0.0,20
20,hasMetals,{SubjectEntity} is an [MASK] made of metal center and organic linkers.,0.0,10
